In [3]:
import os
import requests
from datetime import datetime, timedelta
import pandas as pd
import re
import time
import shutil
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from collections import Counter
from bs4 import BeautifulSoup
import platform
hostname = platform.node()

if hostname == "T_800_Model_101":
   directory = "D:/Dropbox/foreign_earnings/"

if hostname == "ECON-STAFF-ASSOC-2022IMAC-H4TJ80ZFQ6X9-2":
   directory = "/Users/gr2757/Dropbox/foreign_earnings/"
   
if hostname == "jeff":
    directory = "C:/Users/gabri/Dropbox/foreign_earnings/"

In [6]:
full_directory = directory + "/data/SEC_Index"

start_date = "2006-09-30"
end_date = "2007-03-31"

rows = []

firms_dir = directory + "data/CIK_related/linked_firms.csv"

firms = pd.read_csv(firms_dir)

firms = firms.dropna(subset=["cik"])
firms["cik"] = firms["cik"].astype(int).astype(str).str.zfill(10)
cik_list = firms["cik"].tolist()

for filename in os.listdir(full_directory):
    file_path = os.path.join(full_directory, filename)
    if os.path.isfile(file_path):
        with open(file_path, "r") as file:
            for line in file:
                line_items = line.split("|")
                
                if len(line_items) == 5:
                    cik = line_items[0].zfill(10)
                    
                    company_name = line_items[1]
                    form_type = line_items[2]
                    date_filed = line_items[3]
                    partial_url = line_items[4]
                    # print(cik, company_name, form_type, date_filed, partial_url)
                    if cik in cik_list:
                        if form_type == "10-K" or form_type == "10-K/A" and start_date <= date_filed <= end_date:
                            print(partial_url)
                            rows.append({
                                "Firm Name": company_name,
                                "CIK": cik,
                                "Form Type": form_type,
                                "url" : "https://www.sec.gov/Archives/" + partial_url
                            })
                        

df = pd.DataFrame(rows)

edgar/data/1000697/0000950135-06-001427.txt

edgar/data/1001288/0000950152-06-001859.txt

edgar/data/1001606/0001104659-06-015817.txt

edgar/data/1003124/0001193125-06-043805.txt

edgar/data/1009626/0000950134-06-000737.txt

edgar/data/1010552/0001193125-06-051060.txt

edgar/data/1011570/0001193125-06-056535.txt

edgar/data/1015780/0001193125-06-045154.txt

edgar/data/101778/0000950129-06-002297.txt

edgar/data/1018099/0001193125-06-054765.txt

edgar/data/1023131/0001047469-06-002689.txt

edgar/data/102729/0001104659-06-016322.txt

edgar/data/102741/0000897101-06-000075.txt

edgar/data/1027443/0001193125-06-043226.txt

edgar/data/103379/0000893220-06-000503.txt

edgar/data/1034670/0001034670-06-000029.txt

edgar/data/1036960/0000950135-06-001517.txt

edgar/data/103730/0001206774-06-000383.txt

edgar/data/1037646/0000950152-06-001354.txt

edgar/data/1041061/0001041061-06-000109.txt

edgar/data/1043604/0000950134-06-004424.txt

edgar/data/1045309/0000950136-06-001861.txt

edgar/data/1045

In [8]:

firms = firms.rename(columns={"cik": "CIK"})
firms = firms[["CIK", "GVKEY"]]
gvkey_url_cik = pd.merge(firms, df, on="CIK", how="left")

gvkey_url_cik = gvkey_url_cik.dropna(subset=["url"])
gvkey_url_cik.to_csv("../data/gvkey_url_cik.csv", index=False)

gvkey_url_cik

gvkey_url_cik.to_csv(directory + "interface/links_10K_full_2006.csv")

In [5]:

# Setup Chrome options
options = webdriver.ChromeOptions()
options.headless = True

# Choose Chrome Browser
driver = webdriver.Chrome(options=options)


directory = "/Users/gr2757/Dropbox/foreign_earnings/data/CORRESP/"
# directory = "D:/Dropbox/foreign_earnings/data/archive_10-K/"

for index, row in gvkey_url_cik.iterrows():
    cik = str(row['CIK']).zfill(10)
    url = row['url']
    gvkey = str(row['GVKEY'])
    # print(cik, gvkey, url)
    # Create a folder named after the GVKEY if it doesn't exist
    gvkey_directory = os.path.join(directory, gvkey)
    os.makedirs(gvkey_directory, exist_ok=True)
    page_content = None
    
    # Use Selenium to navigate to the URL
    try:
        driver.get(url)
        time.sleep(2)
        
        # Extract page content
        page_content = driver.page_source
        
        if page_content is None:
            print("No content found")
            continue
        
        accession_number_match = re.search(r'\/([^\/]+)\.txt$', url)
        accession_number = accession_number_match.group(1)

        # Save the content in the corresponding folder
        file_name = os.path.join(gvkey_directory, f"{accession_number}.txt")
        
        with open(file_name, "w", encoding="utf-8") as file:
            file.write(page_content)
        
    except Exception as e:
        print("didn't run", cik, gvkey, e)


In [8]:
# # Assuming gvkey_url_cik is already defined and populated

# # Choose Chrome Browser
# driver = webdriver.Chrome()

# directory = "/Users/gr2757/Dropbox/foreign_earnings/data/archive_10-Q_full/"

# print(f"Total rows in gvkey_url_cik: {len(gvkey_url_cik)}")

# for index, row in gvkey_url_cik.iterrows():
#     print(f"Processing row {index + 1}/{len(gvkey_url_cik)}")
#     cik = str(row['CIK']).zfill(10)
#     url = row['url']
#     gvkey = str(row['GVKEY'])
#     # print(cik, gvkey, url)
#     # Create a folder named after the GVKEY if it doesn't exist
#     gvkey_directory = os.path.join(directory, gvkey)
#     os.makedirs(gvkey_directory, exist_ok=True)
#     page_content = None
    
#     # Use Selenium to navigate to the URL
#     try:
#         driver.get(url)
#         time.sleep(3)
        
#         # Extract page content
#         page_content = driver.page_source
        
#         if page_content is None:
#             print("No content found")
#             continue
        
#         accession_number_match = re.search(r'\/([^\/]+)\.txt$', url)
#         accession_number = accession_number_match.group(1)

#         # Save the content in the corresponding folder
#         file_name = os.path.join(gvkey_directory, f"{accession_number}.txt")
        
#         with open(file_name, "w", encoding="utf-8") as file:
#             file.write(page_content)
#         print(f"Saved file: {file_name}")
        
#     except Exception as e:
#         print("didn't run", cik, gvkey, e)